<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/ExothermicTubularReactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig

In [3]:
jnp.set_printoptions(precision=3, linewidth=200)

In [21]:
N=50
q=100. #L/min
ca_in = 1. #mol/L
T_in = 350. #K
Vtotal = 100. #L
V = Vtotal/N  #L
rho = 1000. #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750. #K
k0 = 7.2e10 #1/min

UAtot = 9e4 #J/(min K)
UA = UAtot/N

Tc_ss = 300. #K
Ca_ss_guess =  np.full(N, 0.5) #mol/L
T_ss_guess = np.full(N, 350.) #K
tend = 20. #min

In [22]:
def rhs(t, vec):
    Tc=Tc_ss
    ca, T = jnp.split(vec,2)
    dca, dT = jnp.zeros_like(ca), jnp.zeros_like(T)
    k=k0*jnp.exp(-ER/T)
    dca=dca.at[0].set(q*(ca_in - ca[0])/V - k[0]*ca[0])
    dT=dT.at[0].set((rho*q*C*(T_in-T[0]) - UA*(T[0]-Tc) + negHr*V*k[0]*ca[0])/(rho*V*C))

    dca=dca.at[1:].set(q*(ca[:-1] - ca[1:])/V - k[1:]*ca[1:])
    dT=dT.at[1:].set((rho*q*C*(T[:-1]-T[1:]) - UA*(T[1:]-Tc) + negHr*V*k[1:]*ca[1:])/(rho*V*C))
    return jnp.r_[dca, dT]

rhs=jax.jit(rhs)
rhs_jac = jax.jit(jax.jacobian(rhs, argnums=1))

In [23]:

res=sp.integrate.solve_ivp(rhs, (0, tend), np.r_[Ca_ss_guess, T_ss_guess], method='Radau', dense_output=True, jac=rhs_jac)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.297e-02 ...  8.558e+00  2.000e+01]
        y: [[ 5.000e-01  7.327e-01 ...  9.798e-01  9.798e-01]
            [ 5.000e-01  5.627e-01 ...  9.594e-01  9.594e-01]
            ...
            [ 3.500e+02  3.489e+02 ...  3.117e+02  3.117e+02]
            [ 3.500e+02  3.489e+02 ...  3.109e+02  3.109e+02]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7dcc1c93db20>
 t_events: None
 y_events: None
     nfev: 179
     njev: 2
      nlu: 36

In [24]:
tplot=np.linspace(0,tend,20)
cplot, Tplot = np.split(res.sol(tplot),2,axis=0)
fig = make_subplots(rows=1, cols=2)
tank = np.arange(N)
for i, t in enumerate(tplot):
    fig.add_scatter(x=tank, y=cplot[:,i], row=1, col=1, name=f'c{t:0.1f}', legendgroup=t)
    fig.add_scatter(x=tank, y=Tplot[:,i], row=1, col=2, name=f'T{t:0.1f}', legendgroup=t)
fig.update_layout(height=600, width=1000)

In [25]:
res.y[:,-1]

array([  0.98 ,   0.959,   0.939,   0.918,   0.897,   0.876,   0.854,   0.833,   0.811,   0.789,   0.768,   0.746,   0.724,   0.703,   0.682,   0.662,   0.643,   0.624,   0.607,   0.591,   0.576,
         0.563,   0.551,   0.541,   0.532,   0.524,   0.518,   0.512,   0.507,   0.503,   0.5  ,   0.497,   0.494,   0.492,   0.49 ,   0.489,   0.487,   0.486,   0.485,   0.484,   0.483,   0.482,
         0.481,   0.481,   0.48 ,   0.48 ,   0.479,   0.479,   0.478,   0.478, 350.429, 350.867, 351.316, 351.773, 352.237, 352.706, 353.176, 353.644, 354.103, 354.545, 354.96 , 355.334, 355.651,
       355.891, 356.03 , 356.043, 355.902, 355.582, 355.06 , 354.321, 353.359, 352.179, 350.794, 349.228, 347.512, 345.677, 343.757, 341.785, 339.79 , 337.797, 335.827, 333.897, 332.02 , 330.205,
       328.46 , 326.788, 325.194, 323.677, 322.237, 320.874, 319.586, 318.371, 317.226, 316.148, 315.135, 314.183, 313.289, 312.45 , 311.664, 310.928])

In [26]:
rhs(0,res.y[:,-1])

Array([ 2.220e-16, -3.109e-15, -8.882e-16, -1.998e-15, -3.775e-15, -1.332e-15,  2.442e-15, -4.441e-16,  2.331e-14,  5.373e-14,  6.528e-14, -6.284e-14, -5.633e-13, -1.697e-12, -3.265e-12, -3.973e-12,
       -1.035e-12,  8.890e-12,  2.706e-11,  4.927e-11,  6.402e-11,  5.426e-11,  3.146e-12, -9.868e-11, -2.472e-10, -4.229e-10, -5.932e-10, -7.180e-10, -7.567e-10, -6.753e-10, -4.529e-10, -8.555e-11,
        4.104e-10,  9.977e-10,  1.619e-09,  2.204e-09,  2.672e-09,  2.948e-09,  2.972e-09,  2.709e-09,  2.162e-09,  1.371e-09,  4.040e-10, -6.553e-10, -1.733e-09, -2.789e-09, -3.836e-09, -4.955e-09,
       -6.283e-09, -7.999e-09,  3.530e-13, -1.739e-12,  3.163e-12, -2.939e-12,  2.265e-12,  1.330e-13, -1.668e-12,  6.555e-13, -3.103e-12, -8.341e-12, -1.332e-11,  4.184e-12,  9.428e-11,  2.949e-10,
        5.957e-10,  7.948e-10,  4.473e-10, -9.933e-10, -3.753e-09, -7.257e-09, -9.841e-09, -9.166e-09, -3.031e-09,  9.530e-09,  2.764e-08,  4.855e-08,  6.825e-08,  8.238e-08,  8.715e-08,  8.005e-08,
     

In [28]:
res=sp.optimize.root(lambda v: rhs(0., v), res.y[:,-1], jac = lambda v: rhs_jac(0., v), method='lm',options=dict(ftol=1e-15) )

In [29]:
res.fun

array([ 2.220e-16,  6.661e-16,  1.332e-15, -1.776e-15,  2.220e-15, -5.107e-15,  2.665e-15, -2.220e-16, -1.110e-15,  2.442e-15, -7.105e-15,  4.219e-15, -3.775e-15,  4.441e-16, -2.220e-15, -3.109e-15,
        1.665e-15, -5.773e-15,  4.885e-15, -6.661e-16,  2.331e-15, -2.331e-15, -8.882e-16, -2.220e-16,  3.053e-15,  2.276e-15,  1.499e-15, -4.330e-15,  2.831e-15,  1.082e-15, -1.693e-15,  1.110e-16,
        6.384e-16,  1.027e-15, -1.762e-15,  1.499e-15, -6.384e-16, -3.608e-16,  2.914e-16,  5.759e-16,  8.674e-16, -1.402e-15,  2.567e-16, -9.021e-17,  1.353e-16,  6.002e-16,  2.325e-16, -1.058e-15,
       -1.114e-15,  1.013e-15,  3.530e-13,  5.164e-13, -2.025e-12, -1.340e-13,  2.211e-12, -2.168e-12,  1.136e-12,  5.954e-13, -7.554e-13, -2.319e-13, -1.838e-13,  1.177e-12, -2.475e-14,  1.202e-12,
       -3.966e-13, -8.747e-13,  2.264e-12, -1.202e-12,  1.021e-12,  2.533e-13, -5.791e-13, -1.500e-13, -1.336e-12,  1.085e-12,  1.076e-12, -1.215e-12, -1.391e-12,  2.341e-12, -4.641e-13,  3.248e-13,
     